In [1]:
%config Completer.use_jedi = False
%load_ext autoreload
# %reload_ext autoreload
%autoreload 2
%load_ext lab_black

In [20]:
import os
import mne

%matplotlib inline
import numpy as np

In [56]:
base_dir = "/data/BCI_Competition_IV/"
files = [
    "A02T.gdf",
    "A05T.gdf",
    "A08T.gdf",
    "A01T.gdf",
    "A04T.gdf",
    "A07T.gdf",
    "A03T.gdf",
    "A06T.gdf",
    "A09T.gdf",
]

# files = [
#     "A01E.gdf",
#     "A02T.gdf",
#     "A04E.gdf",
#     "A05T.gdf",
#     "A07E.gdf",
#     "A08T.gdf",
#     "A01T.gdf",
#     "A03E.gdf",
#     "A04T.gdf",
#     "A06E.gdf",
#     "A07T.gdf",
#     "A09E.gdf",
#     "A02E.gdf",
#     "A03T.gdf",
#     "A05E.gdf",
#     "A06T.gdf",
#     "A08E.gdf",
#     "A09T.gdf",
# ]

### Data converter

In [63]:
def extract_labels_and_data_in_epochs(filename):
    raw = mne.io.read_raw_gdf(filename)

    ## Events and Epoch extraction ##
    # Find the events time positions
    events, e_id_mapping = mne.events_from_annotations(raw)

    # Pre-load the data

    raw.load_data()

    # Filter the raw signal with a band pass filter in 7-35 Hz

    # raw.filter(7.0, 35.0, fir_design="firwin")

    # Remove the EOG channels and pick only desired EEG channels

    raw.info["bads"] += ["EOG-left", "EOG-central", "EOG-right"]

    picks = mne.pick_types(
        raw.info, meg=False, eeg=True, eog=False, stim=False, exclude="bads"
    )

    # Extracts epochs of 3s time period from the datset into 288 events for all 4 classes

    tmin, tmax = 1.0, 4.0
    # left_hand = 769,right_hand = 770,foot = 771,tongue = 772
    event_id = dict(
        {
            "769": e_id_mapping["769"],
            "770": e_id_mapping["770"],
            "771": e_id_mapping["771"],
            "772": e_id_mapping["772"],
        }
    )

    epochs = mne.Epochs(
        raw,
        events,
        event_id,
        tmin,
        tmax,
        proj=True,
        picks=picks,
        baseline=None,
        preload=True,
        event_repeated="drop",
    )
    #################################
    # Getting labels and changing labels from 7,8,9,10 -> 1,2,3,4
    labels = epochs.events[:, -1] - np.min(epochs.events[:, -1]) + 1
    # print(epochs.events)
    data = epochs.get_data()

    return data, labels

In [ ]:
with open(os.path.join(base_dir, "metadata.csv"), "w") as f:
    f.write("file_name,patient,split,label\n")
    for file in files:
        patient = file[:3]
        split = file[3]
        print(patient, split)

        data, labels = extract_labels_and_data_in_epochs(os.path.join(base_dir, file))

        for i in range(data.shape[0]):
            file_name = f"{patient}_{split}_{labels[i]}_{i}.npy"
            f.write("{},{},{},{}\n".format(file_name, patient, split, labels[i]))

            folder = "train" if split == "T" else "test"
            np.save(os.path.join(base_dir, folder + "/" + file_name), data[i, :, :])

### see the labels

In [ ]:
raw = mne.io.read_raw_gdf(os.path.join(base_dir, files[0]))

In [ ]:
raw.__dict__

In [ ]:
events, s = mne.events_from_annotations(raw)

In [ ]:
s

In [ ]:
events

In [ ]:
raw.ch_names